In [0]:
%restart_python
%pip install boto3
import boto3
import os
from botocore.exceptions import NoCredentialsError
import datetime
import sys
sys.path.insert(0, '/Workspace/Shared')
import etl_helpers
from pyspark.sql.functions import col #lit
from pyspark.sql.types import StringType, TimestampType #IntegerType, BooleanType

tablename = "dimecgroups"
runcycleid = etl_helpers.start_run_cycle(tablename)
os.makedirs("/dbfs/foi/dataload", exist_ok=True)  # make sure directory exists

try:
    query = f"""
        SELECT
            a.teamid AS groupid,
            a.name AS groupname,
            a.name AS tllgroupname,
            a.description AS groupdescription,
            CASE
                WHEN a.isactive = TRUE THEN 'Y'
            ELSE
                'N'
            END AS isactive,
            NULL AS createddate,
            NULL AS modifieddate,
            a.teamid AS groupcode,
            NULL AS deleteflag,
            NULL AS officeid,
            b.email_address AS emailid,
            'FOIMOD' AS sourceoftruth
        FROM foi_mod.operatingteams a
        INNER JOIN foi_mod.operatingteamemails b
        ON a.teamid = b.teamid
        order by a.teamid;  
        """

    # print(query)

    df = spark.sql(query)
    df.show()

    # order of columns here is important!
    df_mapped = df.selectExpr(
            "groupid AS groupid",
            "groupname AS groupname",
            "tllgroupname AS tllgroupname",
            "groupdescription AS groupdescription",
            "isactive AS isactive",
            "createddate AS createddate",
            "modifieddate AS modifieddate",
            "groupcode AS groupcode",
            "deleteflag AS deleteflag",
            "officeid AS officeid",
            "emailid AS emailid",
            "sourceoftruth AS sourceoftruth"
        )
    df_mapped = df_mapped.withColumn("createddate", col("createddate").cast(TimestampType()))
    df_mapped = df_mapped.withColumn("modifieddate", col("modifieddate").cast(TimestampType()))
    df_mapped = df_mapped.withColumn("deleteflag", col("deleteflag").cast(StringType()))
    df_mapped = df_mapped.withColumn("officeid", col("officeid").cast(StringType()))
    df_mapped.show()

    from delta.tables import DeltaTable
    delta_table = DeltaTable.forName(spark, f"hive_metastore.default.{tablename}")
    delta_table.alias("target").merge(
        df_mapped.alias("source"),
        "target.groupid = source.groupid AND target.sourceoftruth = source.sourceoftruth"
    ).whenMatchedUpdate(set = {
        "groupname": "source.groupname",
        "tllgroupname": "source.tllgroupname",
        "groupdescription": "source.groupdescription",
        "isactive": "source.isactive",
        "createddate": "source.createddate",
        "modifieddate": "source.modifieddate",
        "groupcode": "source.groupcode",
        "deleteflag": "source.deleteflag",
        "officeid": "source.officeid",
        "emailid": "source.emailid"
    }).whenNotMatchedInsert(values = {
        "groupid": "source.groupid",
        "groupname": "source.groupname",
        "tllgroupname": "source.tllgroupname",
        "groupdescription": "source.groupdescription",
        "isactive": "source.isactive",
        "createddate": "source.createddate",
        "modifieddate": "source.modifieddate",
        "groupcode": "source.groupcode",
        "deleteflag": "source.deleteflag",
        "officeid": "source.officeid",
        "emailid": "source.emailid",
        "sourceoftruth": "source.sourceoftruth"
    }).execute()

    etl_helpers.end_run_cycle(runcycleid, 't', tablename)
except NoCredentialsError:
    print("Credentials not available")
    etl_helpers.end_run_cycle(runcycleid, 'f', tablename, "Credentials not available")
    dbutils.notebook.exit("Error: Something went wrong.")
except Exception as e:
    if (str(e) == "no changes for today"):
        print("here")
        etl_helpers.end_run_cycle(runcycleid, 't', tablename)
    else:
        print(f"An error occurred: {e}")    
        etl_helpers.end_run_cycle(runcycleid, 'f', tablename, f"An error occurred: {e}")
        dbutils.notebook.exit("Error: Something went wrong.")